#3DCNN용 동영상 전용 데이터 정제 라이브러리가 없어서
2DCNN과 달리 분석용으로 데이터를 변환하고 모델을 생성하였습니다.


In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

import os
import cv2
from tensorflow.keras.preprocessing import image
import tensorflow.keras
import tensorflow as tf

In [5]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization, Dropout, Activation, InputLayer, Dense, Flatten, Conv3D, MaxPooling3D
from keras.models import Sequential


from tensorflow.keras.optimizers import Adam

In [23]:
# opencv 동영상 정보 확인

video = cv2.VideoCapture('data/KETI_SL_0000000001.avi', cv2.IMREAD_GRAYSCALE)

length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 141
width : 1280
height : 720
fps : 29.970029


In [43]:
#폴더 내 훈련용 데이터셋 확인
#샘플용으론 임시적으로 0~9까지의 데이터만 임의로 가져왔습니다.

directory = 'C:/Users/Chicken is Right/Desktop/opencv/data'
file_list = os.listdir(directory)
file_label = [0,1,2,3,4,5,6,7,8,9] # 별도의 csv가 있지만 샘플용으론 라벨용 리스트를 생성하였습니다.

In [46]:
# 3DCNN용 데이터 정제 부분
count = 0
train_data = [] #전체 훈련용 데이터
tentative_data = [] # 훈련용 영상 데이터 6프레임 단위로 저장
test_data = [] #라벨 저장

for i in range(0, len(file_list)) :
    video = cv2.VideoCapture(directory + '/' + file_list[i] )
    label = file_label[i]
    

    while(video.isOpened()):
        ret, image = video.read()

        if (ret is True) :

            video_fps = video.get(cv2.CAP_PROP_FRAME_COUNT)

            if(int(video.get(1)) % 6 == 0 and int(video_fps)-30 > int(video.get(1)) ): #6fps 단위로 불러옴, 공통적으로 들어있는 마지막 1초 정지 영상은 무시함

                #dst = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                dst = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                resize_img = cv2.resize(dst, (225, 225))
                            #cv2.imwrite("%d.jpg" % count, dst)
                #print('Saved frame number :', str(int(video.get(1))))  

                if count < 6 :
                    tentative_data.append(resize_img)
                    count = count +1
                elif count :

                    train_data.append(tentative_data)
                    test_data.append(label)
                    count = 0
                    tentative_data = []

        else:

            break

    video.release()

In [47]:
#numpy array로 변환


train_data = np.asarray(train_data, dtype=np.float32)
test_data = np.asarray(test_data, dtype=np.float32)

In [48]:
train_data.shape

(26, 6, 225, 225, 3)

In [49]:
test_data

array([0., 0., 1., 1., 1., 2., 2., 3., 3., 3., 4., 4., 4., 5., 5., 6., 6.,
       6., 6., 7., 7., 8., 8., 8., 9., 9.], dtype=float32)

In [53]:
# Create the model
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(6, 225, 225,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
#model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
#model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(BatchNormalization(center=True, scale=True))
#model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
#model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_5 (Conv3D)           (None, 4, 223, 223, 32)   2624      
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 2, 111, 111, 32)  0         
 3D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 2, 111, 111, 32)  128       
 hNormalization)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 2, 111, 111, 32)   0         
                                                                 
 flatten_5 (Flatten)         (None, 788544)            0         
                                                                 
 dense_10 (Dense)            (None, 64)               

In [55]:
model.fit(train_data, test_data, epochs=5)

Epoch 1/5
1/1 [==============================] - 32s 32s/step - loss: 231.7502 - accuracy: 0.1154
Epoch 2/5
1/1 [==============================] - 28s 28s/step - loss: 222.4117 - accuracy: 0.1154
Epoch 3/5
1/1 [==============================] - 27s 27s/step - loss: 176.0657 - accuracy: 0.0769
Epoch 4/5
1/1 [==============================] - 27s 27s/step - loss: 145.1125 - accuracy: 0.2692
Epoch 5/5
1/1 [==============================] - 27s 27s/step - loss: 98.4330 - accuracy: 0.4231


In [56]:
model.save('3DCNN.h5')

# 동영상 데이터 전처리 작업 진행중
